In [ ]:
import pandas
import networkx as nx
import numpy
import matplotlib.pyplot as plt
import json
from networkx.drawing.nx_agraph import write_dot, graphviz_layout
from graph_tool.all import *
from collections import defaultdict

In [ ]:
with open("/mnt/c/Users/aotubusen/Documents/DS Projects/future_news_fund/data/interim/britain_place_tree.json") as f:
    data = json.load(f)

# Data inspection

In [ ]:
len(data[0]['contains'])

In [ ]:
data[0]['name']

In [ ]:
tree_dict = defaultdict(None)
for k in range(len(data)):
    
    print("country: {}".format(data[k]['name']))
    tree_dict[data[k]['name']] = []
    for i in range(len(data[k]['contains'])):
        
        region = data[k]['contains'][i]
        tree_dict[data[k]['name']] = region['name']
        
        print('region: {}'.format(region['name']))

        if region['name'] == region['name']:
            for j in range(len(region['contains'])):
                county=region['contains'][j]['name']
                print("county: {}".format(county))
#etc...            

In [ ]:
tree_dict

In [ ]:
data[1].keys()

# Network graph creation

In [ ]:
G = nx.DiGraph()

G.add_node("ROOT")

for i in range(5):
    G.add_node("Child_%i" % i)
    G.add_node("Grandchild_%i" % i)
    G.add_node("Greatgrandchild_%i" % i)

    G.add_edge("ROOT", "Child_%i" % i)
    G.add_edge("Child_%i" % i, "Grandchild_%i" % i)
    G.add_edge("Grandchild_%i" % i, "Greatgrandchild_%i" % i)

# write dot file to use with graphviz
# run "dot -Tpng test.dot >test.png"
write_dot(G,'test.dot')

# same layout using matplotlib with no labels
plt.title('draw_networkx')
pos =graphviz_layout(G, prog='dot')
nx.draw(G, pos, with_labels=False, arrows=True)
plt.show()

In [ ]:
# G = nx.DiGraph()
# G.add_node('UK')

# for k in range(len(data)):
    
#     G.add_node(data[k]['name'])
#     G.add_edge('UK', data[k]['name'])
#     # print("country: {}".format(data[0]['name']))
# #     tree_dict[data[k]['name']] = []
#     for i in range(len(data[k]['contains'])):

#         region = data[k]['contains'][i]

#         G.add_node(region['name'])
#         G.add_edge(data[k]['name'],region['name'])
#     #     print('region: {}'.format(region['name']))

#         if region['name'] == region['name']:

#             for j in range(len(region['contains'])):
#                 county=region['contains'][j]['name']
#                 G.add_node(county)
#                 G.add_edge(region['name'], county)
#     #             print("county: {}".format(county))
# write_dot(G,'test_1.dot')

# plt.figure(figsize=(25,20))
# plt.title('draw_networkx')

# #
# pos =graphviz_layout(G, prog='circo') #circo dot
# # pos = nx.spring_layout(G,k=0.3)
# nx.draw(G, pos, node_color='yellow', node_size = 50, with_labels=False, arrows=True)
# labels = nx.draw_networkx_labels(G, pos, font_size=7)
# plt.savefig("Graph_test_full.jpg")
# # for _,l in labels.items():
# # #     print(l)
# #     l.set_rotation(45)
# plt.show()


In [ ]:
# nx.to_dict_of_dicts(G)

In [ ]:
# nx.to_dict_of_lists(G)

## Main Graph

In [ ]:
G = nx.DiGraph()
i=0
G.add_node('UK')
#district_inner_type = []

for k in range(len(data)):
    if 'contains' in data[k] and (data[k]['type'] == 'COUNTRY'):
        country = data[k]
        G.add_node(country['name']+'_COUNTRY', type_= country['type'], name_=country['name'])
        G.add_edge('UK', country['name']+'_COUNTRY')

        for region in country['contains']:
    #         print(region['name'])

            if 'contains' in region and (region['type'] == 'REGION'):

    #             print(region['name'])
                G.add_node(region['name']+'_REGION', type_= region['type'], name_=region['name'])
                G.add_edge(country['name']+'_COUNTRY',region['name']+'_REGION')
                for county in region['contains']:
    #                 print(county['name'])


                    if 'contains' in county and (county['type'] == 'COUNTY_UNITARY'):

                        G.add_node(county['name']+'_COUNTY', type_= county['type'], name_=county['name'])
                        G.add_edge(region['name']+'_REGION', county['name']+'_COUNTY')
                        for postcode in county['contains']:
    #                         print(county['name'])

                            if 'contains' in postcode and (postcode['type'] == 'POSTCODE_DISTRICT'):

                                G.add_node(postcode['name']+'_POSTCODE', type_= postcode['type'], name_=postcode['name'])
                                G.add_edge(county['name']+'_COUNTY', postcode['name']+'_POSTCODE')
                    
                                for district in postcode['contains']:
                                    if 'contains' in district and (district['type'] == 'DISTRICT_BOROUGH'):
                                        G.add_node(district['name']+'_DISTRICT', type_= district['type'], name_=district['name'])
                                        G.add_edge(postcode['name']+'_POSTCODE', district['name']+'_DISTRICT')
    #                                     print(district['name'])

                                        for extra in district['contains']:

                                            if ((extra['type'] == 'City') or (extra['type'] == 'Hamlet') or (extra['type'] == 'Other Settlement') or (extra['type'] == 'Postcode') or (extra['type'] == 'Suburban Area') or (extra['type'] == 'Town') or (extra['type'] == 'Village')):
                                                G.add_node(extra['name']+'_', type_= extra['type'], name_=extra['name'])
                                                G.add_edge(district['name']+'_DISTRICT', extra['name']+'_')

                                                #district_inner_type.append(extra['type'])
                                            else:
                                                continue


                                    else:
                                        continue

                            else:
                                continue

                    else:
                        continue

            else:
                continue
                
    else:
            continue

Adding weighted 'weight' attribute to each node- e.g. if Linton appears in 4 different districts, it has a weight of 0.25

In [ ]:
for i in G.nodes:
    if i == 'UK':
        G.nodes()[i]['weight'] = 1
        
    else:
        G.nodes()[i]['weight'] = 1/G.in_degree()[i]
#     print(G.in_degree()[i])

---

In [ ]:
len(G.nodes())

In [ ]:
G.nodes()['Linton_']

In [ ]:
# nx.ancestors(G, source='Amber Valley')
# nx.descendants(G, source='Amber Valley')

In [ ]:
##to plot network graph

# write_dot(G,'test_1_full.dot')

# plt.figure(figsize=(25,45))
# plt.title('draw_networkx')

# #
# pos =graphviz_layout(G, prog='dot') #circo dot
# # pos = nx.spring_layout(G,k=0.3)
# nx.draw(G, pos, node_color='yellow', node_size = 50, with_labels=False, arrows=True)
# labels = nx.draw_networkx_labels(G, pos, font_size=7)

# Graph to dict_of_list format

In [ ]:
dict_list_graph = nx.to_dict_of_lists(G)

In [ ]:
# dict_list_graph.keys()